# Football Match Analysis Notebook

## Project Overview
This notebook implements a comprehensive football match analysis system using computer vision and machine learning techniques. The goal is to extract detailed insights from a single image or video frame, including:
- Player and ball tracking
- Team identification
- Ball possession analysis
- Camera movement tracking

## Key Components
- **Object Detection**: Uses YOLOv5 for identifying players, ball, and referees
- **Team Assignment**: Classifies players into teams based on jersey colors
- **Ball Tracking**: Interpolates ball trajectory and determines possession
- **Camera Movement**: Compensates for camera shifts to maintain accurate tracking

## Technical Workflow
1. Load video/image frame
2. Detect and track objects
3. Assign team colors
4. Determine ball possession
5. Analyze player movements
6. Visualize results

## Technical Dependencies
- OpenCV for image processing
- NumPy for numerical operations
- Custom tracking modules for specialized football analysis

## Future Improvements
- Multi-frame video analysis
- More sophisticated team and player tracking
- Advanced ball possession metrics
- Performance optimization

In [52]:
from utils import *
from trackers import *
from team_assigner import *
from player_ball_assigner import *
from camera_movement import *
from view_transformation import *
from speed_and_distance import *
import numpy as np
import cv2

import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"

In [53]:
frames = read_video('trail_videos/video2.mp4') #here you can play with diferent videos
#frames = [cv2.imread('image3.jpg')] #for single images

In [54]:
tracker = Tracker('models/yolov5lu/best.pt')
tracks = tracker.get_object_tracks(frames, read_from_stub=False, stub_path='stubs/track_stubs.pkl')
tracker.add_position_to_tracks(tracks)


0: 384x640 2 balls, 31 players, 1 referee, 22.3ms
1: 384x640 1 ball, 33 players, 1 referee, 22.3ms
2: 384x640 1 ball, 31 players, 1 referee, 22.3ms
3: 384x640 1 ball, 30 players, 1 referee, 22.3ms
4: 384x640 1 ball, 27 players, 1 referee, 22.3ms
5: 384x640 1 ball, 26 players, 1 referee, 22.3ms
6: 384x640 26 players, 1 referee, 22.3ms
7: 384x640 27 players, 1 referee, 22.3ms
8: 384x640 29 players, 1 referee, 22.3ms
9: 384x640 33 players, 1 referee, 22.3ms
10: 384x640 37 players, 1 referee, 22.3ms
11: 384x640 35 players, 1 referee, 22.3ms
12: 384x640 1 ball, 37 players, 1 referee, 22.3ms
13: 384x640 1 ball, 33 players, 1 referee, 22.3ms
14: 384x640 1 ball, 33 players, 1 referee, 22.3ms
15: 384x640 2 balls, 40 players, 1 referee, 22.3ms
16: 384x640 1 ball, 36 players, 1 referee, 22.3ms
17: 384x640 1 ball, 33 players, 1 referee, 22.3ms
18: 384x640 2 balls, 33 players, 1 referee, 22.3ms
19: 384x640 2 balls, 34 players, 1 referee, 22.3ms
Speed: 1.8ms preprocess, 22.3ms inference, 1.3ms post

In [55]:
camera_movement = CameraMovement(frames[0])
frames_camera_movement = camera_movement.get_camera_movement(frames)#, read_from_stub=False, stub_path='stubs/camera_movement_stub.pkl')
camera_movement.adjust__tracks_positions(tracks, frames_camera_movement)

In [56]:
tracks["ball"] = tracker.ball_interpolation(tracks["ball"])

In [57]:
speed_dist = SpeedAndDistance_Estimator()
#speed_dist.add_speed_and_distance(tracks)

In [58]:
team_assigner = TeamAssigner()
team_assigner.assign_team_color(frames[0], tracks["players"][0])

for n_frame, player_track in enumerate(tracks["players"]):
    for player_id, track in player_track.items():
        team = team_assigner.get_player_team(frames[n_frame], track["bbox"], player_id)
        tracks["players"][n_frame][player_id]["team"] = team
        tracks["players"][n_frame][player_id]["team_color"] = team_assigner.team_colors[team]

In [59]:
player_assigner = PlayerBallAssigner()
team_ball_possession = []

for n_frame, player_track in enumerate(tracks["players"]):
    ball_bbox = tracks["ball"][n_frame][1]["bbox"]
    closest_player = player_assigner.assign_ball_to_player(player_track, ball_bbox)
    
    if closest_player != -1:
        tracks["players"][n_frame][closest_player]["ball_possession"] = True
        team_ball_possession.append(tracks["players"][n_frame][closest_player]["team"])
    else:
        team_ball_possession.append(team_ball_possession[-1])
    
team_ball_possession = np.array(team_ball_possession)

In [60]:
output = tracker.draw_annotations(frames, tracks, team_ball_possession)
#output = camera_movement.draw_camera_movement(output, frames_camera_movement)
output = speed_dist.draw_speed_and_distance(output, tracks)

In [61]:
save_video(output, 'output/output_video.mp4')